# Emotions Detection with Transfer Learning

## dependencies

In [8]:
import os
import random
import tensorflow as tf

## datasets; FER2013

In [4]:
# kaggle api configuration
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [5]:
# downloading the fer2013 dataset
!kaggle datasets download -d msambare/fer2013

Dataset URL: https://www.kaggle.com/datasets/msambare/fer2013
License(s): DbCL-1.0
  0% 0.00/60.3M [00:00<?, ?B/s]
100% 60.3M/60.3M [00:00<00:00, 1.12GB/s]


In [6]:
!ls

fer2013.zip  kaggle.json  sample_data


In [ ]:
# unzipping the dataset
!unzip /content/fer2013.zip -d /content/

In [9]:
!ls

fer2013.zip  kaggle.json  sample_data  test  train


## model data directories

In [10]:
# project name
project_name = "Emotion_Detection_FER2013"

# models list
model_names = [
    'Custom_CNN_From_Scratch',
    'Custom_CNN_With_Augmentation',
    'VGG16_Transfer_Learning',
    'ResNet50_Transfer_Learning'
]

# base directory
base_dir = "/content/"

In [11]:
# creating project directory
project_dir = os.path.join(base_dir, project_name)
os.makedirs(project_dir, exist_ok=True)